This notebook fine-tune an instruct version of Mistral 7B on UltraFeedback with TRL's DPO.

More details in this article: [Fine-tune Your Own Instruct Version of Mistral 7B with Direct Preference Optimization (DPO)](https://kaitchup.substack.com/p/fine-tune-your-own-instruct-version)

First, we need all these dependencies:

In [ ]:
!pip install -q -U bitsandbytes
!pip install -q -U transformers
!pip install -q -U peft
!pip install -q -U accelerate
!pip install -q -U datasets
!pip install -q -U trl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 53.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 31.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 104.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 83.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 28.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.6/85.6 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.0/261.0 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.0/124.0 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━

Import all the necessary packages.

In [ ]:
import torch
from datasets import load_dataset
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    AutoTokenizer,
    TrainingArguments,
)
from trl import DPOTrainer

Load the tokenizer and configure padding

In [ ]:
model_name = "mistralai/Mistral-7B-v0.1"
#Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
tokenizer.pad_token = tokenizer.unk_token
tokenizer.pad_token_id =  tokenizer.unk_token_id
tokenizer.padding_side = 'left'

Load the custom UltraFeedback. I explained in this article how I made it. You can see some examples here: [kaitchup/UltraFeedback-prompt-chosen-rejected](https://huggingface.co/datasets/kaitchup/UltraFeedback-prompt-chosen-rejected)

In the appendix section of this notebook (below), I provide the code I used to make this dataset.

In [ ]:
dataset = load_dataset("kaitchup/UltraFeedback-prompt-chosen-rejected")

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/17075 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/895 [00:00<?, ? examples/s]

Load the model that we will train with DPO and prepare it for QLoRA.

In [ ]:
compute_dtype = getattr(torch, "float16")
bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=compute_dtype,
        bnb_4bit_use_double_quant=True,
)
model = AutoModelForCausalLM.from_pretrained(
          model_name, quantization_config=bnb_config, device_map={"": 0}
)
model = prepare_model_for_kbit_training(model)
#Configure the pad token in the model
model.config.pad_token_id = tokenizer.pad_token_id
model.config.use_cache = False # Gradient checkpointing is used by default but not compatible with caching


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Load the reference model used for DPO. The model I used is Mistral 7B fine-tuned (SFT) on ultrachat. I load it from the Hugging Face Hub.

You can find more details on this fine-tuning in this article: [Mistral 7B: Recipes for Fine-tuning and Quantization on Your Computer](https://kaitchup.substack.com/p/mistral-7b-recipes-for-fine-tuning)

In [ ]:
model_ref_name = "mistralai/Mistral-7B-v0.1"
bnb_config_ref = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=compute_dtype,
        bnb_4bit_use_double_quant=True,
)
model_ref = AutoModelForCausalLM.from_pretrained(
          model_ref_name, quantization_config=bnb_config, device_map={"": 0}
)
model_ref = PeftModel.from_pretrained(model_ref, "kaitchup/Mistral-7B-v0.1-SFT-ultrachat")


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Define the configuration of LoRA used to train the model with DPO.

In [ ]:
peft_config = LoraConfig(
        lora_alpha=16,
        lora_dropout=0.05,
        r=16,
        bias="none",
        task_type="CAUSAL_LM",
        target_modules= ['k_proj', 'q_proj', 'v_proj', 'o_proj', "gate_proj", "down_proj", "up_proj"]
)

For this tutorial, I trained for only 100 steps. It takes 4.5 hours using the V100 of Google Colab Pro. You can also use the T4 but it would take around 15 hours.

In [ ]:
training_arguments = TrainingArguments(
        output_dir="./results",
        evaluation_strategy="steps",
        do_eval=True,
        optim="paged_adamw_8bit",
        per_device_train_batch_size=2,
        gradient_accumulation_steps=16,
        per_device_eval_batch_size=2,
        log_level="debug",
        save_steps=10,
        logging_steps=10,
        learning_rate=5e-7,
        eval_steps=20,
        #num_train_epochs=1,
        max_steps=100,
        warmup_steps=20,
        lr_scheduler_type="linear",
)

Start training:

In [ ]:
trainer = DPOTrainer(
    model,
    model_ref,
    args=training_arguments,
    beta=0.1,
    peft_config=peft_config,
    train_dataset=dataset['train'],
    eval_dataset=dataset['test'],
    tokenizer=tokenizer,
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/trl/trainer/dpo_trainer.py:169: UserWarning: When using DPODataCollatorWithPadding, you should set `max_length` in the DPOTrainer's init it will be set to `512` by default, but you should do it yourself in the future.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/dpo_trainer.py:176: UserWarning: When using DPODataCollatorWithPadding, you should set `max_prompt_length` in the DPOTrainer's init it will be set to `128` by default, but you should do it yourself in the future.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/dpo_trainer.py:205: UserWarning: When using DPODataCollatorWithPadding, you should set `remove_unused_columns=False` in your TrainingArguments we have set it for you, but you should do it yourself in the future.
  warnings.warn(
max_steps is given, it will override any value given in num_train_epochs
Currently training with a batch size of: 2
***** Running training *****
  Num examples = 

Step,Training Loss,Validation Loss,Rewards/chosen,Rewards/rejected,Rewards/accuracies,Rewards/margins,Logps/rejected,Logps/chosen,Logits/rejected,Logits/chosen
20,0.796100,0.756272,-0.204051,-0.265590,0.522321,0.061539,-171.314545,-190.165543,-2.950176,-2.906446
40,0.787100,0.750689,-0.189419,-0.259613,0.525670,0.070194,-171.254776,-190.019196,-2.950228,-2.906829
60,0.792500,0.746549,-0.177696,-0.254009,0.527902,0.076313,-171.198746,-189.901993,-2.950177,-2.907035
80,0.753300,0.744203,-0.171570,-0.251619,0.527902,0.080048,-171.174850,-189.840729,-2.950137,-2.907137
100,0.789000,0.743437,-0.169586,-0.250860,0.527902,0.081274,-171.167236,-189.820892,-2.950131,-2.907176


Saving model checkpoint to ./results/checkpoint-10
tokenizer config file saved in ./results/checkpoint-10/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-10/special_tokens_map.json
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
***** Running Evaluation *****
  Num examples = 895
  Batch size = 2
Saving model checkpoint to ./results/checkpoint-20
tokenizer config file saved in ./results/checkpoint-20/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-20/special_tokens_map.json
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:

TrainOutput(global_step=100, training_loss=0.7826796817779541, metrics={'train_runtime': 16800.0722, 'train_samples_per_second': 0.19, 'train_steps_per_second': 0.006, 'total_flos': 0.0, 'train_loss': 0.7826796817779541, 'epoch': 0.19})

#Appendix
Format UltraFeedback for TRL's DPO.

In [ ]:
from datasets import load_dataset

ultrafb = load_dataset('openbmb/UltraFeedback', split='train')

ultrafb = ultrafb.train_test_split(test_size=0.05)

format_ultrafb = dict()
for split in ultrafb:
  format_ultrafb[split] = []
  for i in ultrafb[split]:
    prompt = i['instruction']
    chosen = ""
    rejected = ""
    for outputs in range(len(i['completions'])):
      if i['completions'][outputs]['annotations']['instruction_following']['Rating'] == '5':
        chosen = i['completions'][outputs]['response']
      elif  i['completions'][outputs]['annotations']['instruction_following']['Rating'] == '1':
        rejected = i['completions'][outputs]['response']
    if chosen != '' and rejected != '':
      format_ultrafb[split].append({'prompt': prompt, 'chosen': chosen, 'rejected': rejected})
